In [2186]:
import pandas as pd
import numpy as np

In [2187]:
filename = "../.gitignore/data_a (Florent).csv"
df = pd.read_csv(filename, skipinitialspace=True) #Open the csv file with no space before and after the values

In [2188]:
print(df.columns)
print(f"rows : {df.shape[0]}\ncolumns : {df.shape[1]}")

Index(['Unnamed: 0', 'To rent', 'To sell', 'Price', 'Number of rooms',
       'Living Area', 'Fully equipped kitchen', 'Furnished', 'Open fire',
       'Terrace', 'Area of the terrace', 'Garden', 'Area of the garden',
       'Surface of the land', 'Surface area of the plot of land',
       'Number of facades', 'Swimming pool', 'State of the building',
       'zipcode', 'type'],
      dtype='object')
rows : 78192
columns : 20


In [2189]:
#Duplicates ?
dupl = len(df)-len(df.drop_duplicates())
#There is no duplicates

In [2190]:
#TEST
#Taking all the rent buildings directly from the source
df_rent = df.query("`To rent` == True" )

#Taking only the appartments and Rez-de-chaussée
df_rent_appart = df_rent.query("type == 'Appartement' | type == 'Rez-de-chaussée'").copy()

print(df_rent_appart.shape)

def deleting_col (df, column_names):
    for column_name in column_names:
        del df[column_name]
    return df

column_names = ['Unnamed: 0', 'To rent', 'To sell', 'Fully equipped kitchen', 'Furnished', 'Area of the terrace', 'Area of the garden']
column_names_follow = ['Surface of the land', 'Surface area of the plot of land', 'Number of facades', 'type', 'Number of rooms']
column_names_final = column_names + column_names_follow
deleting_col(df_rent_appart, column_names_final)

(11072, 20)


,Price,Living Area,Open fire,Terrace,Garden,Swimming pool,State of the building,zipcode
1,790.0,140.0,False,False,False,False,\nBon\n,3200
3,520.0,50.0,False,False,False,False,\nBon\n,6900
4,760.0,55.0,False,True,True,False,NaN,1470
5,640.0,NaN,False,False,False,False,\nBon\n,3200
6,660.0,95.0,False,False,False,False,\nExcellentétat\n,4920
...,...,...,...,...,...,...,...,...
78181,475.0,NaN,False,False,False,False,\nBon\n,7000
78182,590.0,NaN,False,True,True,False,NaN,4100
78185,695.0,NaN,False,True,False,False,NaN,9080
78188,590.0,NaN,False,False,False,False,\nExcellentétat\n,4000


In [2191]:
#Creating the column "Province" and deleting the column zipcode

Province = []
for zipcode in df_rent_appart['zipcode']:
    if (zipcode >= 1000) & (zipcode <= 1299): Province.append('Brussels')
    elif (zipcode >= 1300) & (zipcode <= 1499): Province.append('Brabant wallon')
    elif (zipcode >= 2000) & (zipcode <= 2999): Province.append('Anvers')
    elif (zipcode >= 3500) & (zipcode <= 3999): Province.append('Limbourg')
    elif (zipcode >= 4000) & (zipcode <= 4999): Province.append('Liege')
    elif (zipcode >= 5000) & (zipcode <= 5680): Province.append('Namur')
    elif (zipcode >= 6600) & (zipcode <= 6999): Province.append('Luxembourg')
    elif (zipcode >= 8000) & (zipcode <= 8999): Province.append('Flandre occiendentale')
    elif (zipcode >= 9000) & (zipcode <= 9999): Province.append('Flandre orientale')
    elif (zipcode >= 1500) & (zipcode <= 1999) or (zipcode >= 3000) & (zipcode <= 3499): Province.append('Brabant flamand')
    elif (zipcode >= 6000) & (zipcode <= 6599) or (zipcode >= 7000) & (zipcode <= 7999): Province.append('Hainaut')

df_rent_appart['Province'] = Province
del df_rent_appart['zipcode'] #Not needed anymore

In [2192]:
#Using the median value of the price per square m and calculate the living area missing values from the price per square m for
# each province

provinces = df_rent_appart['Province'].unique()
print(provinces)

anvers = df_rent_appart[df_rent_appart['Province'] == 'Anvers']
anvers_living_area_True = anvers[anvers['Living Area'] > 0].copy()

anvers_living_area_True['Price per square m'] = anvers_living_area_True['Price'] / anvers_living_area_True['Living Area']
median_square_m_price = anvers_living_area_True['Price per square m'].median()

anvers_living_area_False = anvers[anvers['Living Area'].isnull()].copy()
anvers_living_area_False['Living Area'] = anvers_living_area_False['Price'] / median_square_m_price

del anvers_living_area_True['Price per square m']

anvers = pd.concat([anvers_living_area_True, anvers_living_area_False], axis=0)
print(anvers.isnull().sum())
    

['Brabant flamand' 'Luxembourg' 'Brabant wallon' 'Liege' 'Anvers'
 'Hainaut' 'Brussels' 'Flandre orientale' 'Namur' 'Flandre occiendentale'
 'Limbourg']
Price                      0
Living Area                0
Open fire                  0
Terrace                    0
Garden                     0
Swimming pool              0
State of the building    569
Province                   0
dtype: int64


In [2193]:
def replace_living_area_missing_value_from_the_median_square_m_price (df, column_name="Province") :
    provinces = df[column_name].unique()
    
    for province in provinces:
        province_df = df[df[column_name] == province]
        province_living_area_true = province_df[province_df['Living Area'] > 0].copy()
        province_living_area_true['Price per square m'] = province_living_area_true['Price'] / province_living_area_true['Living Area']
        median_square_m_price = province_living_area_true['Price per square m'].median()
        province_living_area_false = province_df[province_df['Living Area'].isnull()].copy()
        province_living_area_false['Living Area'] = province_living_area_false['Price'] / median_square_m_price
        if province == "Brabant flamand":
            final_df = pd.concat([province_living_area_true, province_living_area_false], axis=0)
        else :
            final_df = pd.concat([final_df, province_living_area_true, province_living_area_false], axis=0)
    del final_df['Price per square m']
    return final_df

In [2194]:
df_rent_appart = replace_living_area_missing_value_from_the_median_square_m_price(df_rent_appart)
df_rent_appart.isnull().sum()

Price                       0
Living Area                 0
Open fire                   0
Terrace                     0
Garden                      0
Swimming pool               0
State of the building    3132
Province                    0
dtype: int64

In [2195]:
#How many missing values are there ?
print(df_rent_appart.isnull().sum())
print(df_rent_appart.shape)

Price                       0
Living Area                 0
Open fire                   0
Terrace                     0
Garden                      0
Swimming pool               0
State of the building    3132
Province                    0
dtype: int64
(11072, 8)


In [2196]:
"""del df_rent_appart['Number of rooms'] 

#Testing adding it and deleting the rows with the missing value df_rent_appart = df_rent_appart.dropna(subset='Numbers of rooms')
#df_rent_appart = df_rent_appart.dropna(subset='Number of rooms')

df_rent_appart = df_rent_appart.dropna(subset = 'Living Area')
df_rent_appart = df_rent_appart.dropna(subset = 'State of the building')
print(f"There are {df_rent_appart.shape[0]} rows left")




#Creating the column "Province"

df_rent_appart[(df_rent_appart["zipcode"] >= 1000) & (df_rent_appart["zipcode"] <= 1299)]

Province = []
for zipcode in df_rent_appart['zipcode']:
    if (zipcode >= 1000) & (zipcode <= 1299): Province.append('Brussels')
    elif (zipcode >= 1300) & (zipcode <= 1499): Province.append('Brabant wallon')
    elif (zipcode >= 2000) & (zipcode <= 2999): Province.append('Anvers')
    elif (zipcode >= 3500) & (zipcode <= 3999): Province.append('Limbourg')
    elif (zipcode >= 4000) & (zipcode <= 4999): Province.append('Liege')
    elif (zipcode >= 5000) & (zipcode <= 5680): Province.append('Namur')
    elif (zipcode >= 6600) & (zipcode <= 6999): Province.append('Luxembourg')
    elif (zipcode >= 8000) & (zipcode <= 8999): Province.append('Flandre occiendentale')
    elif (zipcode >= 9000) & (zipcode <= 9999): Province.append('Flandre orientale')
    elif (zipcode >= 1500) & (zipcode <= 1999) or (zipcode >= 3000) & (zipcode <= 3499): Province.append('Brabant flamand')
    elif (zipcode >= 6000) & (zipcode <= 6599) or (zipcode >= 7000) & (zipcode <= 7999): Province.append('Hainaut')

df_rent_appart['Province'] = Province
del df_rent_appart['zipcode'] #Not needed anymore
df_rent_appart = df_rent_appart.dropna(subset="Price")
df_rent_appart = df_rent_appart.drop(columns=["Number of rooms", "Furnished", "Fully equipped kitchen"])
df_rent_appart = df_rent_appart.drop(columns= ['Area of the terrace', 'Area of the garden', 'Surface of the land', 'Surface area of the plot of land'])
df_rent_appart = df_rent_appart.drop(columns= ['Number of facades'])

df_rent_appart = df_rent_appart[df_rent_appart['Price'] < 900000]
print(df_rent_appart['To rent'].unique())
print(df_rent_appart[df_rent_appart['To rent'] == False])
#print(df_rent_appart.head(10))
#print(df_rent_appart.isnull().sum())
#print(df_rent_appart.columns)

#print(df_rent_appart.nlargest(20, 'Price'))"""

print(df_rent_appart.columns)

df_rent_appart = pd.get_dummies(data = df_rent_appart, columns = ['Province', 'State of the building'])









#Modelization part for renting appart in Belgium with LinearRegression

#Defining the features and the target

X = np.array(df_rent_appart.drop(columns = 'Price')) #features = Number of rooms, Living area and zipcode
y = np.array(df_rent_appart['Price']) #target = Renting price

#print(f"That's a sample line for the X_features : {X}")

#Splitting the data between the training part and the testing part
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test =  train_test_split(X,y, test_size = 0.2, random_state = 42 )


#Calling the algorithm we want to apply on our model and applying it
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(X_train, y_train)

#Asking for the score of our model for its training part
training_score = regressor.score(X_train, y_train)
print(f"The score of the model for training part : {training_score}")


#Making a prediction with our model on the testing data part

regressor.predict(X_test)

#Asking the score of our model for its testing part
test_score = regressor.score(X_test, y_test)
print(f"The score of the model for testing part : {test_score}")

Index(['Price', 'Living Area', 'Open fire', 'Terrace', 'Garden',
       'Swimming pool', 'State of the building', 'Province'],
      dtype='object')
The score of the model for training part : 0.10172318178660011
The score of the model for testing part : 0.4262335561027999


In [2197]:
#Deleting the 3992 rows on 78192 who have not the rent or sell info and who haven't any price
df = df.dropna(subset=['Price'])

#Deleting the column "surface of the land", "Surface area of the plot of land"
df = df.drop(columns=['Surface of the land', 'Surface area of the plot of land'])

#Deleting the number of facades column and Area of the garden column and Area of the terrace

df = df.drop(columns=['Number of facades', 'Area of the garden', 'Area of the terrace'])

#Deleting the fully equipped kitchen, furnished and number of rooms columns
df = df.drop(columns=['Fully equipped kitchen', 'Furnished'])

#TEST
#Deleting the column State of the building (Have to also try with deleting the 21643 rows)
#df = df.drop(columns='State of the building')

df_rent = df[df['To rent'] == True]

df_rent_appart = df_rent[(df_rent['type'] == 'Appartement') | (df_rent['type'] == 'Rez-de-chaussée')]

#df_rent_appart = df_rent_appart.dropna(subset="Living Area") #Deleting the empty cells of leaving area

df_rent_appart.isnull().sum()

df_rent_appart = df_rent_appart.drop(columns=['To rent', 'To sell', 'type', "Number of rooms"])

df_rent_appart = df_rent_appart.dropna(subset='State of the building')

df_rent_appart = df_rent_appart.dropna(subset="Living Area")

#df_rent_appart = df_rent_appart.dropna(subset="Number of rooms")


#Checking the numbers of empty cells
df_rent_appart.isnull().sum()
#df_rent_appart.shape


Unnamed: 0               0
Price                    0
Living Area              0
Open fire                0
Terrace                  0
Garden                   0
Swimming pool            0
State of the building    0
zipcode                  0
dtype: int64

In [2198]:
#Creating the column "Province"

df_rent_appart[(df_rent_appart["zipcode"] >= 1000) & (df_rent_appart["zipcode"] <= 1299)]

Province = []
for zipcode in df_rent_appart['zipcode']:
    if (zipcode >= 1000) & (zipcode <= 1299): Province.append('Brussels')
    elif (zipcode >= 1300) & (zipcode <= 1499): Province.append('Brabant wallon')
    elif (zipcode >= 2000) & (zipcode <= 2999): Province.append('Anvers')
    elif (zipcode >= 3500) & (zipcode <= 3999): Province.append('Limbourg')
    elif (zipcode >= 4000) & (zipcode <= 4999): Province.append('Liege')
    elif (zipcode >= 5000) & (zipcode <= 5680): Province.append('Namur')
    elif (zipcode >= 6600) & (zipcode <= 6999): Province.append('Luxembourg')
    elif (zipcode >= 8000) & (zipcode <= 8999): Province.append('Flandre occiendentale')
    elif (zipcode >= 9000) & (zipcode <= 9999): Province.append('Flandre orientale')
    elif (zipcode >= 1500) & (zipcode <= 1999) or (zipcode >= 3000) & (zipcode <= 3499): Province.append('Brabant flamand')
    elif (zipcode >= 6000) & (zipcode <= 6599) or (zipcode >= 7000) & (zipcode <= 7999): Province.append('Hainaut')

df_rent_appart['Province'] = Province
del df_rent_appart['zipcode'] #Not needed anymore

In [2199]:
df_rent_appart.head(10)

,Unnamed: 0,Price,Living Area,Open fire,Terrace,Garden,Swimming pool,State of the building,Province
1,2718948,790.0,140.0,False,False,False,False,\nBon\n,Brabant flamand
3,3860205,520.0,50.0,False,False,False,False,\nBon\n,Luxembourg
6,5123126,660.0,95.0,False,False,False,False,\nExcellentétat\n,Liege
7,5360484,925.0,100.0,False,True,False,False,\nFraîchementrénové\n,Brabant flamand
14,6431847,1650.0,84.0,False,True,False,False,\nExcellentétat\n,Anvers
15,6431856,1650.0,84.0,False,True,False,False,\nExcellentétat\n,Anvers
17,6787654,860.0,95.0,False,True,False,False,\nExcellentétat\n,Hainaut
30,7295023,950.0,80.0,False,False,False,False,\nExcellentétat\n,Brussels
33,7368251,810.0,108.0,False,True,False,False,\nExcellentétat\n,Liege
34,7562816,1350.0,56.0,False,True,False,False,\nExcellentétat\n,Anvers


In [2200]:
df_rent_appart.columns
df_rent_appart.drop(columns=['Unnamed: 0', 'Open fire', 'Terrace', 'Garden', 'Swimming pool'])

,Price,Living Area,State of the building,Province
1,790.0,140.0,\nBon\n,Brabant flamand
3,520.0,50.0,\nBon\n,Luxembourg
6,660.0,95.0,\nExcellentétat\n,Liege
7,925.0,100.0,\nFraîchementrénové\n,Brabant flamand
14,1650.0,84.0,\nExcellentétat\n,Anvers
...,...,...,...,...
78172,650.0,58.0,\nFraîchementrénové\n,Hainaut
78175,800.0,50.0,\nExcellentétat\n,Brussels
78176,850.0,115.0,\nExcellentétat\n,Limbourg
78178,670.0,70.0,\nExcellentétat\n,Hainaut


In [2201]:
df_rent_appart = df_rent_appart[df_rent_appart['Price'] <= 8000] #Deleting the 120120 and 66000 price for renting !
df_rent_appart = df_rent_appart[df_rent_appart['Price'] >= 275] #Deleting the price 45 for renting 89 square m...

#df_rent_appart.nlargest(20, 'Living Area')

df_rent_appart = df_rent_appart.query("`Living Area` <= 550") #Deleting the Living Area > 550 square m
print(df_rent_appart.shape)
del df_rent_appart['Unnamed: 0']
df_rent_appart.columns

(6662, 9)


Index(['Price', 'Living Area', 'Open fire', 'Terrace', 'Garden',
       'Swimming pool', 'State of the building', 'Province'],
      dtype='object')

In [2202]:
df_rent_appart = pd.get_dummies(data = df_rent_appart, columns = ['Province', 'State of the building'])

In [2203]:
#Modelization part for renting appart in Belgium with LinearRegression

#Defining the features and the target

X = np.array(df_rent_appart.drop(columns = 'Price')) #features = 
y = np.array(df_rent_appart['Price']) #target = Renting price

#print(f"That's a sample line for the X_features : {X}")

#Splitting the data between the training part and the testing part
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test =  train_test_split(X,y, test_size = 0.2, random_state = 42 )


#Calling the algorithm we want to apply on our model and applying it
#from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

#regressor = LinearRegression()
regressor = GradientBoostingRegressor()
regressor.fit(X_train, y_train)

#Asking for the score of our model for its training part
training_score = regressor.score(X_train, y_train)
print(f"The score of the model for training part : {training_score}")


#Making a prediction with our model on the testing data part

regressor.predict(X_test)

#Asking the score of our model for its testing part
test_score = regressor.score(X_test, y_test)
print(f"The score of the model for testing part : {test_score}")

The score of the model for training part : 0.7506434004759788
The score of the model for testing part : 0.7653299093788488
